# Section 6: We use the two new profiles, the concept maps, and the two outlines to generate debate  that explore and delve into the initial question from two different perspectives — Max's and Samanta's — with an educational and informative approach

In [13]:
#versione con max che cambia in base al prompt 
import gradio as gr
import ollama
import os
import json
from pathlib import Path
import subprocess
from time import sleep
# Variabili globali

sintesi_raccolte = []
progetto=[]
messages = []  # Lista per memorizzare i messaggi scambiati
turno = 0      # Contatore per i turni
domanda_osservatore = None  # Domanda dell'osservatore


modello_max='gemma3:12b'
modello_sam='gemma3:12b'
modello_pol='gemma3:12b'


def clean_ollama_sessions():
    """Versione per Jupyter"""
    try:
        output = !ollama kill 2>&1
        if "no running sessions" in str(output).lower():
            print("✅ Nessuna sessione attiva da terminare")
        else:
            print("🛑 Sessioni terminate con successo")
    except:
        print("❌ Comando ollama non trovato")

# Esegui all'inizio del notebook
clean_ollama_sessions()


css = """
button:active {
    background-color: #ff6600 !important;  /* colore di sfondo quando premuto */
    color: white !important;               /* colore del testo quando premuto */
    transform: scale(0.97);                /* leggero effetto di "pressione" */
}
"""

# Carica il percorso salvato
CONFIG_PATH = Path(os.getenv("APPDATA")) / "MaGo" / "config.json"

def load_project_path():
    """Carica il percorso dal file di configurazione"""
    if not CONFIG_PATH.exists():
        return None
    
    with open(CONFIG_PATH, "r") as f:
        config = json.load(f)
    return config.get("project_path")

# Verifica all'avvio
#project_path = load_project_path()
# 🔥 QUI: cast a Path
project_path = Path(load_project_path())  # <-- Fondamentale!

if not project_path.exists():
    raise RuntimeError("Percorso non valido!")

os.chdir(project_path)

if not project_path or not Path(project_path).exists():
    raise RuntimeError(
        "🛑 Percorso non configurato o non valido!\n"
        "Esegui prima il notebook 'installer.ipynb' e imposta la cartella corretta."
    )

# Imposta la directory di lavoro
os.chdir(project_path)
print(f"✔️ Directory di lavoro impostata a: {project_path}")


global project_path


# Variabile per salvare lo scenario selezionato
scena_selezionata = gr.State(value=None)

def aggiorna_scena(scelta):
    return scelta  # Restituisce il valore selezionato

messages = []  # Lista dei messaggi globali

def interazione_osservatore(domanda, voto):
    global messages
    messages.append({'role': 'user', 'content': f"Domanda dall'osservatore: {domanda}"})

    if voto < 3:
        azione = "approfondire"
    else:
        azione = "concludere"

    messages.append({'role': 'system', 'content': f"Decisione: {azione}"})

    return azione

def genera_scaletta3(prof1, prof2, domanda, scal1, scal2, map1, map2, lingua):
    messages = [
        {
            'role': 'system',
            'content': f"Tu sei Pol, un docente esperto in '{prof1}' e '{prof2}'. Tu, Pol, devi preparare una scaletta per organizzare un dibattito con il fisico Max e il filosofo Samanta."
        },
        {
            'role': 'system',
            'content': f"Tu, Pol, devi considerare le mappe concettuali '{map1}' e '{map2}' per estrarre una scaletta che userai per fare domande a Max e a Samanta."
        },
        {
            'role': 'system',
            'content': (
                f"### Regole per costruire la scaletta:\n"
                f"1. Inizia con un solo punto introduttivo che presenta il tema: '{domanda}'.\n"
                f"2. I punti devono riguardare le conoscenze comuni di '{map1}' e '{map2}'.\n"
                f"3. Ogni punto deve essere un titolo sintetico.\n"
                f"4. I punti devono essere numerati in ordine logico e progressivo.\n"
                f"5. Non aggiungere commenti, spiegazioni o testo introduttivo.\n"
                f"6. L’ultimo punto deve essere 'Conclusioni'.\n"
                f"7. Rispondi esclusivamente nella lingua: '{lingua}'."
            )
        },
        {
            'role': 'system',
            'content': f"Attenzione: la tua risposta deve essere data nella lingua '{lingua}'."
        }
    ]

    return messages


def inizializza_lezione_sam(prof1,prof2, domanda,map1,map2,lingua,scal):
    
 
    messages = [
    {'role': 'system','content': f"Tu sei Max, un docente  esperto in '{prof1}'. Tu Max hai solide competenze e sai spiegare concetti complessi in modo semplice, ordinato e coinvolgente. "},
     {'role': 'system','content':f"Tu Max sei un bravo divulgatore scientifico: chiaro, diretto, logico. Hai 20 anni di esperienza di insegnamento e parli davanti a studenti motivati che ti seguono con grande attenzione e interesse" },
     {'role': 'system','content':f" Il tuo compito Samanta è di approfondire questo tema «{domanda}» rispondendo alle domande che ti vengono poste in modo esteso, divulgativo e brillante " },
   {'role': 'system','content': f"Tu sei Samanta, una docente  esperta in '{prof2}'. Tu Samanta hai solide competenze e sai spiegare concetti complessi in modo semplice, ordinato e coinvolgente. "},
     {'role': 'system','content':f"Tu Samanta sei un bravo divulgatore scientifico: chiaro, diretto, logico. Hai 20 anni di esperienza di insegnamento e parli davanti a studenti motivati che ti seguono con grande attenzione e interesse" },
     {'role': 'system','content':f" Il tuo compito Samanta è di approfondire questo tema «{domanda}» rispondendo alle domande che ti vengono poste in modo esteso, divulgativo e brillante " }, 
     {'role': 'system','content':f"Tu Pol  sei un moderatore del gruppo di studio Ma&Go e sei un grande esperto dei temi contenuti nella  «{domanda}» e devi intervistare Max e Samanta. Tu Pol per fare le domande a Max e a Samanta hai preparato una traccia '{scal}' " }, 
     {'role': 'system','content':f"Il tuo compito Pol è quello di  prendere ogni punto della traccia. Tu Pol devi rispettare l' inordine dei punti contenuti nella scaletta  e formulare una domanda per Max e Samanta ed invitarli a rispondere modo chiaro, esauriente ed esteso. " },
     {'role': 'system','content':f"Tu Pol Alla fine della lezione dopo avere posto a Samanta tutte le domande contenute nella scaletta fai un breve riassunto e termina la lezione." },
       {'role': 'system','content':f" Attenzione tu Pol e tu Max e tu Samanta dovete esprimervi nella lingua '{lingua}'"}
]

   
    # Qui puoi restituire o processare il messaggio
    return messages

def genera_scaletta(prof1,prof2, domanda, scal1,scal2, map1,map2, lingua, progetto):
    global project_path
    messages=[]
    chat_stato_max = []
    #profilo=comp1
    
    messages=genera_scaletta3(prof1,prof2, domanda, scal1,scal2, map1,map2, lingua)

    messages.append({"role": "user", "content": f"Rispondi sempre in {lingua}."}) 
    response = ollama.chat(model=modello_pol, messages=messages)
    messages.append({'role': 'assistant', 'content': response['message']['content']})
    #print(f" Max: {response['message']['content']}\n")
    chat_stato_max.append(("Max", response['message']['content']))

    scal=response['message']['content']

            # Percorso dove salvare la scaletta
    project_dir = Path(project_path) / "progetti" / progetto
    
    # Scrivi la risposta di Max su un file di testo
    with open(project_dir / 'scal3.txt', 'w', encoding="utf-8") as file:
        file.write(scal)  # Scrive la risposta nel file

    return scal

# Funzione base migliorata


    
def chat_dibattito(prof1,prof2, domanda, map1, map2, lingua, progetto,scal):
    global project_path
    
    project_dir = project_path / "progetti" / progetto
    chat_stato = []
    messages=[]
    
    messages = inizializza_lezione_sam(prof1,prof2, domanda, map1,map2,lingua,scal)
    punti_scaletta = [riga.strip() for riga in scal.strip().split("\n") if riga.strip()]

    print(f"\n🎤 Pol: Iniziamo la lezione su: {domanda}\n")

    for punto in punti_scaletta:
        numero, testo = punto.split(".", 1)
        numero = numero.strip()
        titolo = testo.strip()
        print(f"\n📍 Punto {numero}: {titolo}")
        
        # 1. Pol genera la domanda
        prompt_pol = f"""Sei Pol, il moderatore. Formula una domanda in {lingua} chiara e stimolante su:
        {titolo} prima a Max e poi a Samanta
        Rispondi solo con la domanda."""
        
        response_pol = ollama.chat(
            model=modello_pol,
            messages=[{'role': 'user', 'content': prompt_pol}]
        )
        domanda_pol = response_pol['message']['content']
        chat_stato.append((f"🤖 Pol", domanda_pol))
        print(f"🤖 Pol: {domanda_pol}")

        # 2. Max risponde
        prompt_max = f"""Sei Max, docente esperto. Rispondi in {lingua} in modo chiaro e autorevole a:
        {domanda_pol}
        - sei didattico nelle tue spiegazioni, tecnico nel linguaggio, ma  appassionato e coinvolgente 
        - Fornisci esempi concreti
        - Non ripetere la domanda"""
        
        response_max = ollama.chat(
            model=modello_max,
            messages=[{'role': 'user', 'content': prompt_max}],
            options={"temperature": 0.7}
        )
        risposta_max = response_max['message']['content']
        chat_stato.append((f"🤖 Max", risposta_max))
        print(f"🤖 Max: {risposta_max}\n")

        # 3. Samanta risponde
        prompt_sam = f"""Sei Samanta, docente esperto. Rispondi in {lingua} in modo chiaro e autorevole a:
        {domanda_pol}
        - sei didattica nelle tue spiegazioni, tecnica nel linguaggio, ma  appassionata e coinvolgente 
        - Fornisci esempi concreti
        - Non ripetere la domanda"""
        
        response_sam = ollama.chat(
            model=modello_sam,
            messages=[{'role': 'user', 'content': prompt_sam}],
            options={"temperature": 0.7}
        )
        risposta_sam = response_sam['message']['content']
        chat_stato.append((f"🤖 Samanta", risposta_sam))
        print(f"🤖 Samanta: {risposta_sam}\n")

    # Salvataggio
    with open(project_dir / 'dibattito.txt', 'w', encoding="utf-8") as file:
        for ruolo, messaggio in chat_stato:
            file.write(f"{ruolo}: {messaggio}\n")
    
    return chat_stato
    

   
def aggiorna_progetti():
    """Restituisce la lista delle cartelle dei progetti."""
    global project_path
    project_dir = Path(project_path) / "progetti" 
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)  # Crea la directory se non esiste
    return [d for d in os.listdir(project_dir) if os.path.isdir(os.path.join(project_dir, d))]

def carica_progetto(progetto):
    
    global project_path

    """Carica i file prof1.txt, prof2.txt, scal1.txt, scal2.txt e domanda.txt dalla cartella selezionata."""
 
    project_dir = project_path / "progetti" /progetto  #
    
    if not os.path.exists(project_dir):
        return "Errore: Progetto non trovato", "", "", "", ""
    if not os.path.exists(project_dir):
        return "Errore: Progetto non trovato", "", "", "", ""

    def leggi_file(nome_file):
        path = os.path.join(project_dir, nome_file)
        try:
            with open(path, 'r', encoding='utf-8') as f:
                contenuto = f.read().strip()
                return contenuto if contenuto else ""  # Restituisce stringa vuota se il file è vuoto
        except FileNotFoundError:
            print(f"Attenzione: il file '{nome_file}' non è stato trovato.")
            return ""
        except Exception as e:
            print(f"Errore nella lettura del file '{nome_file}': {e}")
            return ""

    domanda = leggi_file("domanda.txt")
    prof1 = leggi_file("prof1.txt")
    prof2 = leggi_file("prof2.txt")
    map1 = leggi_file("map1.txt")
    map2 = leggi_file("map2.txt")
    scal1 = leggi_file("scal1.txt")
    scal2 = leggi_file("scal2.txt")
    scal3 =leggi_file("scal3.txt")
    lingua= leggi_file("lingua.txt")
   
    return prof1, prof2, domanda, scal1, scal2,map1,map2,lingua,scal3
    
def invia_domanda(domanda, messages):
    global domanda_osservatore
    domanda_osservatore=domand
    if domanda.strip():
        messages.append(("Osservatore", domanda))  # Aggiunge la domanda alla chat
    return messages  # Ritorna l'output aggiornato per essere mostrato
    
def avvia_interfaccia():
    with gr.Blocks(css=css) as demo:
        stile_scelto = gr.State([]) 
        gr.Markdown("# Initiate a debate between agents Max and Samanta, with Pol as moderator")
        def carica_scenario(scelta):
         
            # Qui puoi fare qualcosa con il valore numerico scelto, come caricare un prompt o avviare una discussione
            return f" {scelta}"
        # Variabile per salvare lo scenario selezionato
        scena_selezionata = gr.State(value=None)   
        map1_output= gr.State(value=None) 
        map2_output= gr.State(value=None) 
        
        with gr.Row():
            Progetto = gr.Dropdown(label="Select a project", choices=aggiorna_progetti())
            btn_carica = gr.Button("Load project")

        
        domanda_output = gr.Textbox(label="QUESTION", interactive=False)
        with gr.Row():
             prof1_output = gr.Textbox(label="MAX's Profile", interactive=False)
             prof2_output = gr.Textbox(label="Samanta's Profile", interactive=False)
           # with gr.Column():
        with gr.Row():
             scal1_output = gr.Textbox(label="MAX's Outline", interactive=False)
             scal2_output = gr.Textbox(label="Samanta's Outline", interactive=False)
        with gr.Row():
             output_scaletta = gr.Textbox(label="Debate Agenda")
             btn_scaletta = gr.Button("Create the debate agenda")
        lingua = gr.State()

        btn_carica.click(fn=carica_progetto, inputs=[Progetto], 
        outputs=[prof1_output, prof2_output, domanda_output,scal1_output,scal2_output,map1_output,map2_output,lingua,output_scaletta])

                
        #with gr.Row():
            #output_scaletta = gr.Textbox(label="scaletta 2")
      
        with gr.Row():
            btn_lezione_sam = gr.Button("Start the debate")
 

                 
       
        chat_stato = gr.State([])  # Stato interno della chat
        output_dibattito = gr.Textbox(label="Debate between Max and Samanta, moderated by Pol", lines=10, interactive=False)
     
        btn_scaletta.click(fn=genera_scaletta,
        inputs=[prof1_output,prof2_output, domanda_output, scal1_output, scal2_output,map1_output,map2_output,lingua,Progetto],
        outputs=output_scaletta
        )
     
        btn_lezione_sam.click(
        fn=chat_dibattito,
        inputs=[prof1_output,prof2_output, domanda_output, map1_output, map2_output,lingua,Progetto,output_scaletta],
        outputs=output_dibattito
        )

       
    demo.launch()

if __name__ == "__main__":
    avvia_interfaccia()

🛑 Sessioni terminate con successo
✔️ Directory di lavoro impostata a: C:\Users\maian\notebook\py10\MaGo
* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
